In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random as rand

from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df = pd.read_excel('./documents/aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21:22].to_numpy()

In [4]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [5]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [6]:
def my_loss(real_data, gen_data):
    dist = tf.math.sqrt(tf.reduce_sum(tf.math.square(tf.math.subtract(real_data, gen_data))))
    y_pred = dnn_model(gen_data)

    a = tf.cast(tf.math.subtract(y_pred, 42), dtype=tf.float32)
    b = tf.cast(dist, dtype=tf.float32)

    loss = tf.math.add(a, b)
    return loss

In [7]:
input_layer = tf.keras.layers.Input(shape=(21,))

encoded = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(64, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(32, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(16, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(3)(encoded)
 
decoded = tf.keras.layers.Dense(3, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(16, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(32, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(64, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(128, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(21, activation='sigmoid')(decoded)
 
# autoencoder
autoencoder = tf.keras.Model(input_layer, decoded)
 
# encoder
encoder = tf.keras.Model(input_layer, encoded)

# decoder
encoded_input = tf.keras.layers.Input(shape=(3,))

decoded_layer = autoencoder.layers[6](encoded_input)
decoded_layer = autoencoder.layers[7](decoded_layer)
decoded_layer = autoencoder.layers[8](decoded_layer)
decoded_layer = autoencoder.layers[9](decoded_layer)
decoded_layer = autoencoder.layers[10](decoded_layer)
decoded_layer = autoencoder.layers[11](decoded_layer)

decoder = tf.keras.Model(encoded_input, decoded_layer)

In [8]:
autoencoder.compile(optimizer='adam', loss=my_loss, metrics=['mse'])

In [9]:
result = []
pop_result = []
for i in range(1000):
    X_train = X[rand.randint(0, 394)].reshape(-1, 1).T
    X_pred = autoencoder.predict(X_train, verbose=0)

    loss = my_loss(X_train, X_pred)
    autoencoder.train_on_batch(X_pred, X_train)

    pop = dnn_model.predict(X_pred, verbose=0)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[0.749676]], pop: [[41.03061]]
epoch: 1, loss: [[-0.13604295]], pop: [[40.937767]]
epoch: 2, loss: [[-0.17323828]], pop: [[40.852768]]
epoch: 3, loss: [[-0.21752518]], pop: [[40.829758]]
epoch: 4, loss: [[-0.25902557]], pop: [[40.752472]]
epoch: 5, loss: [[-0.12513864]], pop: [[40.698215]]
epoch: 6, loss: [[-0.23125756]], pop: [[40.604675]]
epoch: 7, loss: [[-0.01333976]], pop: [[40.591366]]
epoch: 8, loss: [[-0.78545195]], pop: [[40.31992]]
epoch: 9, loss: [[-0.20241511]], pop: [[40.21931]]
epoch: 10, loss: [[-0.2583232]], pop: [[40.225174]]
epoch: 11, loss: [[-0.961882]], pop: [[39.941963]]
epoch: 12, loss: [[-0.90184677]], pop: [[39.627636]]
epoch: 13, loss: [[-1.0137517]], pop: [[39.461823]]
epoch: 14, loss: [[-1.82941]], pop: [[39.116543]]
epoch: 15, loss: [[-1.9038304]], pop: [[38.7324]]
epoch: 16, loss: [[-2.875444]], pop: [[38.048702]]
epoch: 17, loss: [[-2.9645765]], pop: [[37.82092]]
epoch: 18, loss: [[-3.6103563]], pop: [[37.34442]]
epoch: 19, loss: [[-4.206

In [10]:
gen = autoencoder.predict(X)

13/13 [==============================] - 0s 1ms/step


In [11]:
print(dnn_model.predict(gen, verbose=0).max())
print(dnn_model.predict(gen, verbose=0).min())
print(dnn_model.predict(gen, verbose=0).mean())

25.818327
25.71476
25.728006


In [12]:
latest = np.array([X[-1]])
latest_gen = autoencoder.predict(latest)
dist = np.sqrt(np.sum(np.square(latest-latest_gen)))

print(dist)

1/1 [==============================] - 0s 21ms/step
1.7540607785424058


In [13]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [14]:
df = np.append(X_result, pop_result, axis=1)

In [15]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/ae_formula.xlsx')